<a href="https://colab.research.google.com/github/CL500Coupe/data_analytics_assi1/blob/main/assi2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**imports**

In [67]:
import sys
import os
import pandas as pd
import seaborn as sns
import pickle
import time
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
!pip install -U keras-tuner
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch
from sklearn.metrics import accuracy_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


**final pre-processing in order to work with models**

In [35]:
url = "https://raw.githubusercontent.com/CL500Coupe/data_analytics_assi1/main/data/final_data.csv"
data = pd.read_csv(url)

In [36]:
# Convert the 'collision_date' column to datetime format
data['collision_date'] = pd.to_datetime(data['collision_date'])

# Set the 'collision_date' column as the index
data = data.set_index('collision_date')

In [37]:
# List of columns with NA values to fill with mean values of the last 7 available days
columns_with_na_7 = [
    "visib_7day_ma",
    "wdsp_7day_ma",
    "co2_7day_ma",
    "so2_7day_ma",
    "pressure_7day_ma",
    "temp_7day_ma",
    "wind_7day_ma",
]

# List of columns with NA values to fill with mean values of the last 30 available days
columns_with_na_30 = [
    "visib_30day_ma",
    "wdsp_30day_ma",
    "co2_30day_ma",
    "so2_30day_ma",
    "pressure_30day_ma",
    "temp_30day_ma",
    "wind_30day_ma",
]

# Fill NA values with the mean values of the first 7 available days
for col in columns_with_na_7:
    first_valid_index = data[col].first_valid_index()
    mean_value = data.loc[first_valid_index:first_valid_index + pd.Timedelta(days=6), col].mean()
    data[col] = data[col].fillna(mean_value, limit=6)

# Fill NA values with the mean values of the first 30 available days
for col in columns_with_na_30:
    first_valid_index = data[col].first_valid_index()
    mean_value = data.loc[first_valid_index:first_valid_index + pd.Timedelta(days=29), col].mean()
    data[col] = data[col].fillna(mean_value, limit=29)

# Display the updated DataFrame
print(data.head())



                day  year  mo  temp  dewp     slp  visib  wdsp  mxpsd  gust  \
collision_date                                                                
2012-07-01        7  2012   7  83.6  63.0  1008.9    9.7   4.1    9.9  18.1   
2012-07-02        1  2012   7  80.3  54.1  1011.6   10.0   3.8   15.0  18.1   
2012-07-03        2  2012   7  79.8  56.7  1012.8   10.0   2.9   12.0  18.1   
2012-07-04        3  2012   7  81.8  65.6  1009.1    9.2   3.6   11.1  18.1   
2012-07-05        4  2012   7  86.7  64.3  1007.4    9.4   3.8   15.0  18.1   

                ...  co2_30day_ma  so2_30day_ma  pressure_30day_ma  \
collision_date  ...                                                  
2012-07-01      ...     15.031652      1.210648         917.765355   
2012-07-02      ...     15.031652      1.210648         917.765355   
2012-07-03      ...     15.031652      1.210648         917.765355   
2012-07-04      ...     15.031652      1.210648         917.765355   
2012-07-05      ...     15

In [38]:
data.shape


(2969, 39)

In [39]:
data.head()

,day,year,mo,temp,dewp,slp,visib,wdsp,mxpsd,gust,...,co2_30day_ma,so2_30day_ma,pressure_30day_ma,temp_30day_ma,wind_30day_ma,temp_visib_interaction,wdsp_visib_interaction,prcp_temp_interaction,temp_season_interaction,wdsp_wind_interaction
collision_date,,,,,,,,,,,,,,,,,,,,,
2012-07-01,7,2012,7,83.6,63.0,1008.9,9.7,4.1,9.9,18.1,...,15.031652,1.210648,917.765355,76.092,93.001803,810.92,39.77,0.000,167.2,553.145520
2012-07-02,1,2012,7,80.3,54.1,1011.6,10.0,3.8,15.0,18.1,...,15.031652,1.210648,917.765355,76.092,93.001803,803.00,38.00,0.000,160.6,545.418750
2012-07-03,2,2012,7,79.8,56.7,1012.8,10.0,2.9,12.0,18.1,...,15.031652,1.210648,917.765355,76.092,93.001803,798.00,29.00,0.000,159.6,362.080104
2012-07-04,3,2012,7,81.8,65.6,1009.1,9.2,3.6,11.1,18.1,...,15.031652,1.210648,917.765355,76.092,93.001803,752.56,33.12,4.908,163.6,453.314999
2012-07-05,4,2012,7,86.7,64.3,1007.4,9.4,3.8,15.0,18.1,...,15.031652,1.210648,917.765355,76.092,93.001803,814.98,35.72,8669.133,173.4,479.916250


In [40]:
data.dtypes

day                          int64
year                         int64
mo                           int64
temp                       float64
dewp                       float64
slp                        float64
visib                      float64
wdsp                       float64
mxpsd                      float64
gust                       float64
max                        float64
min                        float64
prcp                       float64
fog                          int64
NUM_COLLISIONS               int64
co2                        float64
wind                       float64
so2                        float64
pressure                   float64
season                       int64
visib_7day_ma              float64
wdsp_7day_ma               float64
co2_7day_ma                float64
so2_7day_ma                float64
pressure_7day_ma           float64
temp_7day_ma               float64
wind_7day_ma               float64
visib_30day_ma             float64
wdsp_30day_ma       

In [41]:
data.columns

Index(['day', 'year', 'mo', 'temp', 'dewp', 'slp', 'visib', 'wdsp', 'mxpsd',
       'gust', 'max', 'min', 'prcp', 'fog', 'NUM_COLLISIONS', 'co2', 'wind',
       'so2', 'pressure', 'season', 'visib_7day_ma', 'wdsp_7day_ma',
       'co2_7day_ma', 'so2_7day_ma', 'pressure_7day_ma', 'temp_7day_ma',
       'wind_7day_ma', 'visib_30day_ma', 'wdsp_30day_ma', 'co2_30day_ma',
       'so2_30day_ma', 'pressure_30day_ma', 'temp_30day_ma', 'wind_30day_ma',
       'temp_visib_interaction', 'wdsp_visib_interaction',
       'prcp_temp_interaction', 'temp_season_interaction',
       'wdsp_wind_interaction'],
      dtype='object')

In [42]:
# Check if there are any missing values in the entire DataFrame
has_missing_values = data.isna().any().any()

# Print the result
print("Are there any missing values in the DataFrame?", has_missing_values)

Are there any missing values in the DataFrame? False


In [43]:
# Select the features and target variable
X = data.drop("NUM_COLLISIONS", axis=1)  # Replace "num_collisions" with the actual target column name
y = data["NUM_COLLISIONS"]  # Replace "num_collisions" with the actual target column name

**baseline linear regression model**

In [ ]:
# Create a linear regression model
model = LinearRegression()

In [ ]:
# Compute the cross-validated MSE and MAE scores
mse_scores = -1 * cross_val_score(model, X, y, cv=5, scoring='neg_mean_squared_error')
mae_scores = -1 * cross_val_score(model, X, y, cv=5, scoring='neg_mean_absolute_error')

In [ ]:
# Compute the mean squared error (MSE), mean absolute error (MAE), and their square roots (RMSE, RMAE)
mse = mse_scores.mean()
rmse = mse_scores.mean() ** 0.5
mae = mae_scores.mean()


In [ ]:
print("Mean Squared Error: {:.4f}".format(mse))
print("Root Mean Squared Error: {:.4f}".format(rmse))
print("Mean Absolute Error: {:.4f}".format(mae))

Mean Squared Error: 64576.1269
Root Mean Squared Error: 254.1183
Mean Absolute Error: 156.9246


**linear regression model with hyperparameters tuning**

In [ ]:
params = {
    'alpha': [0.001, 0.01, 0.1, 1, 10],
    'selection': ['cyclic', 'random'],
    'max_iter': [1000, 5000, 10000],
    'tol': [0.0001, 0.001, 0.01],
}

In [ ]:
# Create the Lasso regression model
lasso = Lasso()

In [ ]:
# Create the GridSearchCV object
grid_search = GridSearchCV(lasso, param_grid=params, cv=10)

In [ ]:
# Fit the model using the GridSearchCV object
grid_search.fit(X_train, y_train)

/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.434e+06, tolerance: 3.517e+03
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.413e+05, tolerance: 3.389e+03
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.161e+03, tolerance: 3.500e+03

GridSearchCV(cv=10, estimator=Lasso(),
             param_grid={'alpha': [0.001, 0.01, 0.1, 1, 10],
                         'max_iter': [1000, 5000, 10000],
                         'selection': ['cyclic', 'random'],
                         'tol': [0.0001, 0.001, 0.01]})

In [ ]:
# Print the best parameters found
print('Best Parameters:', grid_search.best_params_)

Best Parameters: {'alpha': 0.001, 'max_iter': 1000, 'selection': 'random', 'tol': 0.001}


In [ ]:
# Make predictions using the best model
y_pred = grid_search.predict(X_test)


In [ ]:
# Compute the mean squared error (MSE), mean absolute error (MAE), and their square roots (RMSE, RMAE)
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
mae = mean_absolute_error(y_test, y_pred)


In [ ]:
print('Mean Squared Error:', mse)
print('Root Mean Squared Error:', rmse)
print('Mean Absolute Error:', mae)

Mean Squared Error: 36669.85895982583
Root Mean Squared Error: 191.49375697349987
Mean Absolute Error: 93.69948218684006


**baseline DNN model**

In [ ]:
# Split the data into training and testing sets
X_train_DNN, X_test_DNN, y_train_DNN, y_test_DNN = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Define the model
model_DNN = Sequential([
    Dense(64, activation='relu', input_shape=[X_train_DNN.shape[1]]),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(1)
])

In [ ]:
# Compile the model
model_DNN.compile(optimizer='adam', loss='mse', metrics=['mae'])

In [ ]:
# Train the model
history = model_DNN.fit(X_train_DNN, y_train_DNN, epochs=50, batch_size=32, validation_split=0.2)

Epoch 1/50
60/60 [==============================] - 1s 5ms/step - loss: 62792.6445 - mae: 185.4000 - val_loss: 20392.2441 - val_mae: 109.2400
Epoch 2/50
60/60 [==============================] - 0s 3ms/step - loss: 28259.7539 - mae: 128.9646 - val_loss: 20176.6113 - val_mae: 102.0436
Epoch 3/50
60/60 [==============================] - 0s 3ms/step - loss: 24857.0820 - mae: 119.8150 - val_loss: 18890.1797 - val_mae: 102.3080
Epoch 4/50
60/60 [==============================] - 0s 3ms/step - loss: 24579.5156 - mae: 118.7704 - val_loss: 19101.5195 - val_mae: 101.7523
Epoch 5/50
60/60 [==============================] - 0s 3ms/step - loss: 22976.0039 - mae: 115.6413 - val_loss: 18659.5586 - val_mae: 100.9188
Epoch 6/50
60/60 [==============================] - 0s 6ms/step - loss: 23786.7598 - mae: 117.1157 - val_loss: 18856.8203 - val_mae: 102.5273
Epoch 7/50
60/60 [==============================] - 0s 8ms/step - loss: 23509.4766 - mae: 119.2188 - val_loss: 18263.3301 - val_mae: 98.6006
Epoch 8

In [ ]:
# Use the trained DNN model to predict on the test set
y_pred = model_DNN.predict(X_test_DNN)

19/19 [==============================] - 0s 1ms/step


In [ ]:
# Calculate Mean Squared Error
mse = mean_squared_error(y_test, y_pred)

In [ ]:
# Calculate Root Mean Squared Error
rmse = np.sqrt(mse)

In [ ]:
# Calculate Mean Absolute Error
mae = mean_absolute_error(y_test, y_pred)

In [ ]:
# Print the evaluation metrics
print("Mean Squared Error: {:.4f}".format(mse))
print("Root Mean Squared Error: {:.4f}".format(rmse))
print("Mean Absolute Error: {:.4f}".format(mae))

Mean Squared Error: 19514.6697
Root Mean Squared Error: 139.6949
Mean Absolute Error: 94.9869


**DNN with hyperparams tuning, layers**

In [14]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
# Define the model builder function
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Dense(units=hp.Int('units', min_value=32, max_value=512, step=32), 
                           activation='relu', 
                           input_shape=X_train.shape[1:]))
    for i in range(hp.Int('num_layers', 1, 5)):
        model.add(layers.Dense(units=hp.Int('units_' + str(i), min_value=32, max_value=512, step=32), 
                               activation='relu'))
        model.add(layers.Dropout(rate=hp.Float('dropout_' + str(i), 0, 0.5, step=0.1)))
    model.add(layers.Dense(1, activation='linear'))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
        loss='mean_squared_error',
        metrics=['mean_absolute_error'])
    return model

In [16]:
# Define the hyperparameter search space
tuner = RandomSearch(
    build_model,
    objective='val_mean_absolute_error',
    max_trials=5,
    executions_per_trial=3,
    directory='my_dir',
    project_name='assi2')

In [17]:
# Perform the hyperparameter search
tuner.search(X_train, y_train, epochs=50, validation_split=0.2)

Trial 5 Complete [00h 01m 54s]
val_mean_absolute_error: 103.4116923014323

Best val_mean_absolute_error So Far: 91.16100311279297
Total elapsed time: 00h 08m 14s


In [18]:
# Print the best hyperparameters found
best_hp = tuner.get_best_hyperparameters()[0]
print(f"Best hyperparameters: {best_hp}")

Best hyperparameters: <keras_tuner.engine.hyperparameters.hyperparameters.HyperParameters object at 0x7fcef9e2c940>


In [20]:
# Print the best hyperparameters found
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print("Best hyperparameters:")
for hp in best_hps.space:
    print(f"{hp.name}: {best_hps.get(hp.name)}")


Best hyperparameters:
units: 192
num_layers: 1
units_0: 384
dropout_0: 0.0
learning_rate: 0.001
units_1: 64
dropout_1: 0.4
units_2: 32
dropout_2: 0.2
units_3: 224
dropout_3: 0.4


In [25]:
# Define the model architecture
model1 = keras.Sequential()
model1.add(layers.Dense(units=192, activation='relu', input_shape=X_train.shape[1:]))
model1.add(layers.Dense(units=384, activation='relu'))
model1.add(layers.Dropout(rate=0.0))
model1.add(layers.Dense(units=64, activation='relu'))
model1.add(layers.Dropout(rate=0.4))
model1.add(layers.Dense(units=32, activation='relu'))
model1.add(layers.Dropout(rate=0.2))
model1.add(layers.Dense(units=224, activation='relu'))
model1.add(layers.Dropout(rate=0.4))
model1.add(layers.Dense(1, activation='linear'))


In [26]:
# Compile the model
optimizer = keras.optimizers.Adam(learning_rate=0.001)
model1.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['mean_absolute_error'])

In [27]:
# Train the model
history = model1.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

Epoch 1/50
60/60 [==============================] - 2s 7ms/step - loss: 66877.6016 - mean_absolute_error: 202.4925 - val_loss: 45954.0625 - val_mean_absolute_error: 191.1412
Epoch 2/50
60/60 [==============================] - 0s 5ms/step - loss: 38701.9102 - mean_absolute_error: 158.4056 - val_loss: 53609.4141 - val_mean_absolute_error: 208.5180
Epoch 3/50
60/60 [==============================] - 0s 5ms/step - loss: 35040.4375 - mean_absolute_error: 147.2077 - val_loss: 80965.7500 - val_mean_absolute_error: 262.4445
Epoch 4/50
60/60 [==============================] - 0s 5ms/step - loss: 30773.5352 - mean_absolute_error: 137.8779 - val_loss: 78369.7969 - val_mean_absolute_error: 258.0372
Epoch 5/50
60/60 [==============================] - 0s 5ms/step - loss: 29571.8555 - mean_absolute_error: 136.8626 - val_loss: 83285.3906 - val_mean_absolute_error: 266.5720
Epoch 6/50
60/60 [==============================] - 0s 5ms/step - loss: 27445.0645 - mean_absolute_error: 131.1923 - val_loss: 841

In [29]:
# Use the trained DNN model to predict on the test set
y_pred = model1.predict(X_test)

19/19 [==============================] - 1s 11ms/step


In [30]:
# Calculate Mean Squared Error
mse = mean_squared_error(y_test, y_pred)


In [31]:
# Calculate Root Mean Squared Error
rmse = np.sqrt(mse)

In [32]:
# Calculate Mean Absolute Error
mae = mean_absolute_error(y_test, y_pred)

In [33]:
# Print the evaluation metrics
print("Mean Squared Error: {:.4f}".format(mse))
print("Root Mean Squared Error: {:.4f}".format(rmse))
print("Mean Absolute Error: {:.4f}".format(mae))

Mean Squared Error: 113215.7035
Root Mean Squared Error: 336.4754
Mean Absolute Error: 315.4475


**Baseline RandomForestClassifier**

In [48]:
# Get the summary statistics of the num_collisions column
summary = data['NUM_COLLISIONS'].describe()

# Print the results
print(summary)


count    2969.000000
mean      573.540923
std       128.360026
min        93.000000
25%       511.000000
50%       587.000000
75%       655.000000
max      1161.000000
Name: NUM_COLLISIONS, dtype: float64


In [57]:
# Create bins for the classes
bins = [0, 100, 150, 200, 250, 300, 350, 400, 450, 500, 550, 600, 650, 700, 750, 800, 850, 900, 950, 1000, np.inf]

# Create the class labels
labels = range(1, len(bins))

# Add the class labels as a new column to the dataframe
data['collision_class'] = pd.cut(data['NUM_COLLISIONS'], bins=bins, labels=labels)

data = data.drop(['NUM_COLLISIONS'],axis=1)

In [58]:
data.head()

,day,year,mo,temp,dewp,slp,visib,wdsp,mxpsd,gust,...,so2_30day_ma,pressure_30day_ma,temp_30day_ma,wind_30day_ma,temp_visib_interaction,wdsp_visib_interaction,prcp_temp_interaction,temp_season_interaction,wdsp_wind_interaction,collision_class
collision_date,,,,,,,,,,,,,,,,,,,,,
2012-07-01,7,2012,7,83.6,63.0,1008.9,9.7,4.1,9.9,18.1,...,1.210648,917.765355,76.092,93.001803,810.92,39.77,0.000,167.2,553.145520,10
2012-07-02,1,2012,7,80.3,54.1,1011.6,10.0,3.8,15.0,18.1,...,1.210648,917.765355,76.092,93.001803,803.00,38.00,0.000,160.6,545.418750,11
2012-07-03,2,2012,7,79.8,56.7,1012.8,10.0,2.9,12.0,18.1,...,1.210648,917.765355,76.092,93.001803,798.00,29.00,0.000,159.6,362.080104,13
2012-07-04,3,2012,7,81.8,65.6,1009.1,9.2,3.6,11.1,18.1,...,1.210648,917.765355,76.092,93.001803,752.56,33.12,4.908,163.6,453.314999,8
2012-07-05,4,2012,7,86.7,64.3,1007.4,9.4,3.8,15.0,18.1,...,1.210648,917.765355,76.092,93.001803,814.98,35.72,8669.133,173.4,479.916250,11


In [53]:
#x = data[['NUM_COLLISIONS','collision_class']]

In [60]:
# Select the features and target variable
X_class = data.drop("collision_class", axis=1)  # Replace "num_collisions" with the actual target column name
y_class = data["collision_class"]  # Replace "num_collisions" with the actual target column name

In [61]:
# Split the data into training and testing sets
X_train_class, X_test_class, y_train_class, y_test_class = train_test_split(X_class, y_class, test_size=0.2, random_state=42)

In [64]:
# Define the classifier with the desired hyperparameters
rf_clf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)

# Train the classifier on the training set
rf_clf.fit(X_train_class, y_train_class)

# Predict on the test set
y_pred_class = rf_clf.predict(X_test_class)

In [65]:
# Print the classification report and confusion matrix
print(classification_report(y_test_class, y_pred_class))
print(confusion_matrix(y_test_class, y_pred_class))

              precision    recall  f1-score   support

           2       1.00      0.50      0.67         6
           3       0.56      0.71      0.63         7
           4       0.40      0.40      0.40         5
           5       0.43      0.30      0.35        10
           6       0.29      0.29      0.29         7
           7       0.50      0.08      0.13        13
           8       0.47      0.28      0.35        25
           9       0.30      0.17      0.22        59
          10       0.31      0.34      0.32        98
          11       0.24      0.30      0.27        94
          12       0.27      0.47      0.34       107
          13       0.28      0.30      0.29        88
          14       0.40      0.10      0.16        41
          15       0.00      0.00      0.00        22
          16       0.00      0.00      0.00         8
          17       0.00      0.00      0.00         2
          18       0.00      0.00      0.00         1
          19       0.00    

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [68]:
accuracy = accuracy_score(y_test_class, y_pred_class)
print(f"Overall accuracy: {accuracy}")

Overall accuracy: 0.29292929292929293


**RandomForestClassfier with hyperparams**

In [69]:
# Define the parameter grid for GridSearchCV
param_grid = {
    'n_estimators': [100, 200, 500],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2']
}

# Create a random forest classifier object
rfc = RandomForestClassifier()

# Create a GridSearchCV object with 5-fold cross-validation
grid_search = GridSearchCV(
    estimator=rfc,
    param_grid=param_grid,
    scoring='accuracy',
    cv=5,
    n_jobs=-1)

# Fit the GridSearchCV object to the data
grid_search.fit(X_train_class, y_train_class)

# Print the best hyperparameters found
best_params = grid_search.best_params_
print(f"Best hyperparameters: {best_params}")

# Get the best estimator
best_rfc = grid_search.best_estimator_

# Get the accuracy score of the best estimator on the test set
accuracy = best_rfc.score(X_test_class, y_test_class)
print(f"Accuracy: {accuracy}")

/usr/local/lib/python3.9/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


Best hyperparameters: {'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 200}
Accuracy: 0.30134680134680136
